In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta

import config

### Get data

In [2]:
trips = pd.read_csv(config.TRIPS_1216_PRC, low_memory=False)
trips.shape

(44351, 17)

In [3]:
trips.head()

,TRIPID,PERSID,HHID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,STARTIME,ARRTIME,TRIPTIME,TRAVTIME,WAITIME,ORIGSA1,DESTSA1,CUMDIST,HomeSA1,Mode
0,Y12H0000107P01T01,Y12H0000107P01,Y12H0000107,1,Sunday,Vehicle Driver,Social,630,650,20,20,0,20904122028,20904122002,2.60,20904122028,Car
1,Y12H0000107P01T02,Y12H0000107P01,Y12H0000107,2,Sunday,Vehicle Driver,Social,750,760,10,10,0,20904122002,20904122028,2.60,20904122028,Car
2,Y12H0000107P01T03,Y12H0000107P01,Y12H0000107,3,Sunday,Walking,Recreational,810,840,30,30,0,20904122028,20904122031,3.23,20904122028,Walk
3,Y12H0000107P01T04,Y12H0000107P01,Y12H0000107,4,Sunday,Walking,Recreational,855,885,30,30,0,20904122031,20904122028,3.23,20904122028,Walk
4,Y12H0000107P02T01,Y12H0000107P02,Y12H0000107,1,Sunday,Vehicle Passenger,Social,630,650,20,20,0,20904122028,20904122002,2.60,20904122028,Car


In [4]:
trips['Mode'].value_counts()

Walk     20277
Car      19823
Train     2171
Cycle     2080
Name: Mode, dtype: int64

In [5]:
choice_mapping = {'Walk':1, 'Car':2, 'Train':3, 'Cycle':4}
trips['CHOICE'] = trips.Mode.map(choice_mapping)

In [7]:
# Only select these columns for MNL model
trips = trips[['TRIPTIME', 'TRAVTIME', 'WAITIME', 'CUMDIST','CHOICE']]

### Simple MNL model

In [8]:
database = db.Database('melbourne_mnl', trips)
globals().update(database.variables)

In [9]:
### Parameters to be estimatd:
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 1)

B_TIME = Beta('B_TIME', 0, None, None, 0)

In [10]:
### Utility function
V_WALK = ASC_WALK + B_TIME * TRIPTIME
V_CAR = ASC_CAR + B_TIME * TRIPTIME
V_TRAIN = ASC_TRAIN + B_TIME * TRIPTIME
V_CYCLE = ASC_CYCLE + B_TIME * TRIPTIME

V = { 1: V_WALK, 2: V_CAR, 3: V_TRAIN, 4: V_CYCLE }

### Avaiability: Assume 1 for all alternatives
av = {1:1, 2:1, 3:1, 4:1}

In [11]:
### Create logit model
logprob = models.loglogit(V, av, CHOICE)
biogeme = bio.BIOGEME(database, logprob)

In [12]:
%%time
### Estimate paramaters
results = biogeme.estimate()

CPU times: user 1.5 s, sys: 6.67 ms, total: 1.51 s
Wall time: 328 ms


In [13]:
### Get results
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,2.254587e+00,0.023049,9.781597e+01,0.000000,0.023050,9.781108e+01,0.000000
ASC_TRAIN,4.295330e-02,0.030683,1.399918e+00,0.161538,0.030683,1.399886e+00,0.161547
ASC_WALK,2.277238e+00,0.023025,9.890392e+01,0.000000,0.023026,9.889900e+01,0.000000
B_TIME,-1.536751e-10,21427.548110,-7.171847e-15,1.000000,0.000684,-2.247730e-07,1.000000


In [102]:
results.getGeneralStatistics()

{'Number of estimated parameters': (4, ''),
 'Sample size': (44351, ''),
 'Excluded observations': (0, ''),
 'Init log likelihood': (-61483.54121003348, '.7g'),
 'Final log likelihood': (-44747.17349584851, '.7g'),
 'Likelihood ratio test for the init. model': (33472.73542836994, '.7g'),
 'Rho-square for the init. model': (0.2722089096496896, '.3g'),
 'Rho-square-bar for the init. model': (0.2721438515882755, '.3g'),
 'Akaike Information Criterion': (89502.34699169702, '.7g'),
 'Bayesian Information Criterion': (89537.14655383861, '.7g'),
 'Final gradient norm': (0.16863009221099626, '.4E'),
 'Nbr of threads': (8, '')}

In [103]:
results.getCorrelationResults()

,Covariance,Correlation,t-test,p-value,Rob. cov.,Rob. corr.,Rob. t-test,Rob. p-value
ASC_TRAIN-ASC_CAR,0.000481,6.798823e-01,-97.831810,0.000000,0.000481,0.679908,-97.832812,0.000000
ASC_WALK-ASC_CAR,0.000481,9.060093e-01,2.267807,0.023341,0.000481,0.906019,2.267812,0.023341
ASC_WALK-ASC_TRAIN,0.000481,6.806064e-01,98.943218,0.000000,0.000481,0.680632,98.944266,0.000000
B_TIME-ASC_CAR,-0.000013,-2.664800e-08,-0.000105,0.999916,-0.000013,-0.847842,-95.400482,0.000000
B_TIME-ASC_TRAIN,-0.000013,-2.010091e-08,-0.000002,0.999998,-0.000013,-0.628876,-1.380340,0.167482
B_TIME-ASC_WALK,-0.000014,-2.747455e-08,-0.000106,0.999915,-0.000014,-0.876344,-96.381482,0.000000
